In [1]:
import os
import json
import uuid
import logging
from pathlib import Path
from typing import List, Dict, Any, TypedDict, Optional
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field, ValidationError
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END

In [2]:
def create_jd_extraction_chain():
    """Create LLM chain for extracting JD criteria."""
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "Extract key criteria from the job description in JSON format."),
            ("human", "Analyze this JD:\n\n{jd}"),
        ]
    )
    return prompt | llm.with_structured_output(JDCriteria)

In [3]:
def create_pii_redaction_chain():
    """Create LLM chain for PII redaction."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Remove all PII (names, emails, phones, addresses, universities). "
                "Replace names with [CANDIDATE] and universities with [UNIVERSITY]. "
                "Return only the redacted text without any explanation.",
            ),
            ("human", "Redact this resume:\n\n{resume_text}"),
        ]
    )
    return prompt | llm

In [5]:
def create_evaluation_chain():
    """Create LLM chain for candidate evaluation."""
    prompt = ChatPromptTemplate.from_template(
        """Evaluate this candidate against the job criteria.

            Job Requirements:
            - Skills: {required_skills}
            - Experience: {experience} years
            
            Redacted Resume:
            {resume_text}
            
            Provide evaluation in JSON format with overall_score (1-100), summary, and skill_justifications."""
    )
    return prompt | llm.with_structured_output(CandidateEvaluation)

In [6]:
def unwrap_structured(
    response: Any, expected_type: type, context: str = ""
) -> Optional[Any]:
    """
    Safely unwrap and validate structured LLM output.

    Args:
        response: The LLM response to validate
        expected_type: Expected Pydantic model type
        context: Context string for error logging

    Returns:
        Validated model instance or None on failure
    """
    try:
        if isinstance(response, expected_type):
            return response
        elif isinstance(response, dict):
            return expected_type(**response)
        else:
            logger.error(f"{context}: Unexpected response type: {type(response)}")
            return None
    except ValidationError as e:
        logger.error(f"{context}: Validation error: {e}")
        return None
    except Exception as e:
        logger.error(f"{context}: Unexpected error: {e}")
        return None